In [1]:
import numpy as np 
import pandas as pd 
import random
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
p = .3
df_main = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_main

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,All Be Okay,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,"Blackbird - Live at CitiField, NYC - Digital A...",HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Dance Tonight,HARD ROCK 2010
...,...,...,...,...
3870199,2302bf9c64dc63d88a750215ed187f2c,Soundgarden,The Day I Tried To Live,iPhone
3870200,2302bf9c64dc63d88a750215ed187f2c,Jimi Hendrix,Voodoo Child (Slight Return),iPhone
3870201,2302bf9c64dc63d88a750215ed187f2c,Ratt,Wanted Man,iPhone
3870202,2302bf9c64dc63d88a750215ed187f2c,Metallica,Welcome Home (Sanitarium),iPhone


In [3]:
df_main.columns = df_main.columns.str.replace('"', '')
df_main.columns = df_main.columns.str.replace('name', '')
df_main.columns = df_main.columns.str.replace(' ', '')
df_main.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

# Data Transformation and Pre-processing

In [4]:
df_main = df_main.groupby('artist').filter(lambda x : len(x)>=50)

In [5]:
df_main = df_main[df_main.groupby('user_id').artist.transform('nunique') >= 10]

In [6]:
size = lambda x: len(x)
df_freq = df_main.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
950593,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,1022
168448,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,1011
281683,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,780
7361,014e695cc6df96011b90a5beb3206012,Ilaiyaraaja,689
927106,d993ff8f2de226e2c6803e47a22e9d7e,Lata Mangeshkar,683


In [7]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Ilaiyaraaja
2,2,Lata Mangeshkar
3,3,Grateful Dead
4,4,Wolfgang Amadeus Mozart


In [8]:
df_freq  = pd.merge(df_freq , df_artist, how='inner', on='artist')
df_freq.head()

,user_id,artist,freq,artist_id
0,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,1022,0
1,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,1011,0
2,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,780,0
3,5ed466280594117226d840fed87a8bc6,Vitamin String Quartet,92,0
4,e78e1e7b93c32bc27bf458f6cb8a5554,Vitamin String Quartet,90,0


In [9]:
df_pivot = df_freq.pivot(index='user_id',columns='artist_id',values='freq')
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,9823,9824,9825,9826,9827,9828,9829,9830,9831,9832
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0007f3dd09c91198371454c608d47f22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c11a16c89aa4b14b328080f5954ee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00123e0f544dee3ab006aa7f1e5725a7,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00139e9cb50fb309549e1561b476226d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe11226cdea81a2db9262c0ec7f5d71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe32d5412269f3041c58cbf0dde3306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff60baf392613ed33f745b89a9b38f7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_pivot = df_pivot.fillna(0)
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,9823,9824,9825,9826,9827,9828,9829,9830,9831,9832
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0007f3dd09c91198371454c608d47f22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c11a16c89aa4b14b328080f5954ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00123e0f544dee3ab006aa7f1e5725a7,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00139e9cb50fb309549e1561b476226d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe11226cdea81a2db9262c0ec7f5d71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffe32d5412269f3041c58cbf0dde3306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff60baf392613ed33f745b89a9b38f7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model Training

In [11]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors = 50,metric = 'cosine')
Model = knn.fit(df_pivot)

In [12]:
def KNN_recommend(index):
    user_id = df_pivot.index[index]
    user = df_pivot.iloc[index,]
    distances, indices = Model.kneighbors([user])
    neighbors=[]
    for item in indices[0][1:]:
        try:
            neighbors.append(df_main[df_main.index == item]["user_id"].values[0])
        except IndexError:
            continue
    neighbors = list(set(neighbors))
    df_neighbor = pd.DataFrame(columns=['user_id', 'artist','freq','artist_id'])
    for item in neighbors:
        df_neighbor = df_neighbor.append(df_freq[df_freq.user_id == item],ignore_index = True)
    df_neighbor = df_neighbor.sort_values('freq',ascending=False)
    df_neighbor_subset = pd.DataFrame({'artist':df_neighbor['artist'], 'freq':df_neighbor['freq']})
    df_neighbor_subset = df_neighbor_subset.groupby('artist').sum()
    df_neighbor_subset.sort_values('freq',ascending=False, inplace = True)
    df_neighbor_subset = df_neighbor_subset.reset_index()
    user_playlists = []
    for artist in df_freq.loc[df_freq.user_id == user_id]['artist']:
        user_playlists.extend(df_main.loc[df_main['artist'] == artist].playlist.squeeze().unique())
    target_artists = []
    for playlist in user_playlists[:10]:
        target_artists.extend(df_main.loc[df_main['playlist'] == playlist].artist.squeeze())
    target_artists = list(set(target_artists))
    size = lambda x: len(x)
    df_playlist_count = df_main.groupby(['playlist']).agg('size').reset_index().rename(columns={0:'freq'})[['playlist', 'freq']].sort_values(['freq'], ascending=False)
    df_playlist_count = df_playlist_count.reset_index(drop = True)
    top_100 = df_playlist_count[0:len(df_playlist_count)//3].sample(n = 100)
    y_artist = df_main[df_main['playlist'].isin(top_100["playlist"])].artist.squeeze().unique()
    return len(list(set(y_artist) & set(df_neighbor_subset.artist[0:10]))) * 10, df_neighbor_subset.artist[0:10] # Converting Accuracy to percentage


# Model Evaluation

In [13]:
accuracy = []
for index in range(100):
    accuracy.append(KNN_recommend(index)[0])
print(sum(accuracy) / 100)

72.9


### Using KNN, the model accuracy is calculated to be 72.9%.

# Output

In [14]:
# Recommend 10 songs for a particular user
recommended_artists = KNN_recommend(index)[1]
recommended_artists = pd.DataFrame(recommended_artists)[0:5] # Top 5 recommended artists
recommended_artists

,artist
0,Nine Inch Nails
1,Ed Sheeran
2,Pearl Jam
3,Kent
4,Foo Fighters


In [15]:
final_recommendations = {}
for artist in recommended_artists['artist']:
    final_recommendations[artist] = df_main[df_main['artist'] == artist]['track'].value_counts()[0:5].index
final_recommendations = pd.DataFrame(final_recommendations)
final_recommendations.T # Top recommended 5 songs of every recommended artist

,0,1,2,3,4
Nine Inch Nails,Closer,Hurt,The Hand That Feeds,Came Back Haunted,March Of The Pigs
Ed Sheeran,Thinking Out Loud,The A Team,I See Fire,Sing,Don't
Pearl Jam,Even Flow,Alive,Jeremy,Just Breathe,Black
Kent,Utan dina andetag,999,La Belle Epoque,Pärlor,Dom Andra
Foo Fighters,Everlong,Best Of You,The Pretender,My Hero,Learn to Fly
